In [ ]:
import itertools
import math
import re
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.rcParams["figure.figsize"] = (16, 6)
# plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams["figure.constrained_layout.use"] = True  # Experimental

sns.set_style("white")
sns.set_context("notebook")

In [ ]:
# ESC to plot
# also defines plotting order (first to last = left to right)
esc_order = [
    "hvdc-to-h2",
    "pipeline-h2",
    "shipping-lh2",
    "shipping-lohc",
    "pipeline-ch4",
    "shipping-lch4",
    "shipping-lnh3",
    "shipping-meoh",
    "shipping-ftfuel",
]

# exporter colors and prettier ESC names from config
exp_colors = snakemake.config["plotting"]["exporter_colors"]
esc_pretty_names = snakemake.config["plotting"]["esc_pretty_names"]

# Select data of interest
df = pd.read_csv(snakemake.input[0], sep=";")

df = df.loc[
    (df["scenario"] == snakemake.wildcards["scenario"])
    & (df["year"] == int(snakemake.wildcards["year"]))
    & (df["subcategory"] == "Cost per MWh delivered")
]

# Only ESCs of interest
df = df.loc[df["esc"].isin(esc_order)]

# Sort dataframe for ordered plotting
df = df.sort_values(by=["esc", "value"])


## Width and positioning of bars
# number of ESCs determines label position anchor
xs = np.arange(df["esc"].unique().size)
# number of exporters determines label position modifier
xxs = np.arange(df["exporter"].unique().size)
# bar widths, used to calculate positions
width = 0.1


## Prepare figure canvas and add figure; one bar at a time
fig = plt.figure(figsize=(12, 4))
ax = plt.gca()

for (esc, x) in zip(esc_order, xs):

    esc_data = df[df["esc"] == esc][["exporter", "value"]]

    for ((idx, row), xx) in zip(esc_data.iterrows(), xxs):
        ax.bar(
            x + xx * width - (xxs.shape[0] - 1) * width / 2,
            row["value"],
            width,
            color=exp_colors[row["exporter"]],
        )

## Construct legend manually
# First entry (legend title)
legend_exp = [
    matplotlib.lines.Line2D(
        [],
        [],
        color="None",
        linestyle="None",
        markersize=0,
        label="Exporter",
        marker=None,
    )
]
# All other entries (exporter labels and color markers)
legend_exp += [
    matplotlib.lines.Line2D(
        [], [], linestyle="None", markersize=15, marker=".", label=n, color=c
    )
    for n, c in exp_colors.items()
]
# Add legend to plot
ax.legend(handles=legend_exp, ncol=9, loc="upper center")

# Add named xtick labels, one per group (nicely named ESC labels)
ax.set_xticks(xs)
ax.set_xticklabels([esc_pretty_names[esc] for esc in esc_order])

# ytick labels
ax.set_ylabel("Cost per energy delivered [EUR/MWh]")

# Grid lines / style
ax.grid(which="major", axis="y", visible=True, lw=2)
ax.grid(which="minor", axis="y", visible=True, alpha=0.5)
ax.minorticks_on()

# Save figure for all file types (PDF, PNG, ...)
for fp in snakemake.output["figures"]:
    plt.savefig(fp, dpi=snakemake.config["plotting"]["dpi"])